<a href="https://colab.research.google.com/github/krcpr007/Watermarking-Deep-Neural-Networks/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import Regularizer
import tensorflow.keras.backend as K
from scipy.special import comb
import tensorflow as tf
def compute_mismatch_threshold(C=10, Kp=50, p=0.05):
    prob_sum = 0
    p_err = 1 - 1.0 / C
    for i in range(Kp):
        cur_prob = comb(Kp, i, exact=False) * np.power(p_err, i) * np.power(1 - p_err, Kp - i)
        prob_sum = prob_sum + cur_prob
        if prob_sum > p:
            theta = i
            break

    return theta

def key_generation(x_train, y_train, marked_model, desired_key_len, num_classes=10, embed_epoch=20, modulation_strength=60000):
    key_len = np.dot(40, desired_key_len)
    batch_size = 128
    num_classes = 10
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)  # Updated to tf.keras.utils
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)    # Updated to tf.keras.utils
    key_gen_flag = 1
    while key_gen_flag:
        np.random.seed()
        x_retrain_rand = np.random.randint(256, size=(key_len, 784))
        x_retrain_rand = x_retrain_rand / 255.0
        np.random.seed()
        y_retrain_rand_vec = np.random.randint(10, size=(key_len, 1))
        y_retrain_rand = tf.keras.utils.to_categorical(y_retrain_rand_vec, num_classes)  # Updated to tf.keras.utils
        x_train_subset = x_train[0:modulation_strength, :]
        y_train_subset = y_train[0:modulation_strength, :]
        x_retrain = np.vstack((x_train_subset, x_retrain_rand))
        y_retrain = np.vstack((y_train_subset, y_retrain_rand))
        unmarked_score = marked_model.evaluate(x_test, y_test, verbose=0)
        prediction_random_key = marked_model.predict(x_retrain_rand, batch_size=batch_size)
        preds = np.argmax(prediction_random_key, axis=1)
        preds = np.reshape(preds, (key_len, 1))
        mismatched_result = (preds != y_retrain_rand_vec) * 1
        random_unmarkMismatched_idx = np.argwhere(mismatched_result)
        random_unmarkMismatched_idx = random_unmarkMismatched_idx[:, 0]
        history = marked_model.fit(x_retrain, y_retrain, batch_size=batch_size, epochs=embed_epoch, shuffle=True, verbose=1, validation_data=(x_test, y_test))
        score = marked_model.evaluate(x_test, y_test, verbose=0)
        score = marked_model.evaluate(x_retrain_rand, y_retrain_rand, verbose=0)
        Perr_marked = 1 - score[1]
        mark_NN_err = int(Perr_marked * key_len)
        prediction_random_key = marked_model.predict(x_retrain_rand, batch_size=batch_size)
        preds = np.argmax(prediction_random_key, axis=1)
        preds = np.reshape(preds, (key_len, 1))
        matched_result = (preds == y_retrain_rand_vec) * 1
        matched_result = np.reshape(matched_result, (matched_result.shape[0], 1))
        random_MarkMatched_idx = np.argwhere(matched_result)
        random_MarkMatched_idx = random_MarkMatched_idx[:, 0]
        selected_key_idx = np.intersect1d(random_MarkMatched_idx, random_unmarkMismatched_idx)
        selected_keys = x_retrain_rand[np.array(selected_key_idx).astype(int), :]
        selected_keys_labels = y_retrain_rand[np.array(selected_key_idx).astype(int)]
        usable_key_len = selected_keys.shape[0]
        print('Usable key len is: ', usable_key_len)
        if usable_key_len < desired_key_len:
            key_gen_flag = 1
            print(' Desired key length is {}, Longer key needed, skip this test. '.format(desired_key_len))
        else:
            key_gen_flag = 0
            selected_keys = selected_keys[0:desired_key_len, :]
            selected_keys_labels = selected_keys_labels[0:desired_key_len]
            np.save('/content/drive/MyDrive/Colab Notebooks/keyRandomImage' + '_keyLength' + str(desired_key_len) + '.npy', selected_keys)
            np.savetxt('/content/drive/MyDrive/Colab Notebooks/keyRandomLabel' + '_keyLength' + str(desired_key_len) + '.txt', selected_keys_labels, fmt='%i', delimiter=',')
            actual_key_len = selected_keys.shape[0]
            marked_model.save_weights('/content/drive/MyDrive/Colab Notebooks/markedWeights' + '.h5')
            print('WM key generation finished. Save watermarked model. ')
            break

    return (selected_keys, selected_keys_labels)

def count_response_mismatch(Y_preds, Y_key):
    num_mismatch = np.sum((Y_preds == Y_key) * 1)
    return num_mismatch


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, SGD
import tensorflow.keras.backend as K
import numpy as np

def create_model(num_classes=10):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary() #shows summery of model

    return model


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
import tensorflow.keras.backend as K
from tensorflow.keras.regularizers import Regularizer
import numpy as np

def initial_conv(input):
    x = Conv2D(16, (3, 3), padding="same")(input)

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)
    return x

def conv1_block(input, k=1, dropout=0.0, regularizer=None):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    if init.shape[channel_axis] != 16 * k:
        init = Conv2D(16 * k, (1, 1), activation="linear", padding="same")(init)

    x = Conv2D(16 * k, (3, 3), padding="same")(input)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    if dropout > 0.0:
        x = Dropout(dropout)(x)

    x = Conv2D(16 * k, (3, 3), padding="same", kernel_regularizer=regularizer)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    m = concatenate([init, x], axis=channel_axis)
    return m

def conv2_block(input, k=1, dropout=0.0, regularizer=None):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    if init.shape[channel_axis] != 32 * k:
        init = Conv2D(32 * k, (1, 1), activation="linear", padding="same")(init)

    x = Conv2D(32 * k, (3, 3), padding="same")(input)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    if dropout > 0.0:
        x = Dropout(dropout)(x)

    x = Conv2D(32 * k, (3, 3), padding="same", kernel_regularizer=regularizer)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    m = concatenate([init, x], axis=channel_axis)
    return m

def conv3_block(input, k=1, dropout=0.0, regularizer=None):
    init = input

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    if init.shape[channel_axis] != 64 * k:
        init = Conv2D(64 * k, (1, 1), activation='linear', padding='same')(init)

    x = Conv2D(64 * k, (3, 3), padding='same')(input)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    if dropout > 0.0:
        x = Dropout(dropout)(x)

    x = Conv2D(64 * k, (3, 3), padding='same', kernel_regularizer=regularizer)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    m = concatenate([init, x], axis=channel_axis)
    return m

def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1, wmark_regularizer=None, target_blk_num=1):
    def get_regularizer(blk_num, idx):
        if wmark_regularizer is not None and target_blk_num == blk_num and idx == 0:
            print('target regularizer({}, {})'.format(blk_num, idx))
            return wmark_regularizer
        else:
            return None

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    for i in range(N):
        x = conv1_block(x, k, dropout, get_regularizer(1, i))
        nb_conv += 2

    x = MaxPooling2D((2,2), padding='same')(x)

    for i in range(N):
        x = conv2_block(x, k, dropout, get_regularizer(2, i))
        nb_conv += 2

    x = MaxPooling2D((2,2), padding='same')(x)

    for i in range(N):
        x = conv3_block(x, k, dropout, get_regularizer(3, i))
        nb_conv += 2

    x = AveragePooling2D((8,8), padding='same')(x)
    x = Flatten()(x)

    x = Dense(nb_classes, activation='softmax')(x)

    model = Model(ip, x)

    if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model

if __name__ == "__main__":
    init = (32, 32, 3)

    wrn_28_10 = create_wide_residual_network(init, nb_classes=100, N=4, k=10, dropout=0.25)

    wrn_28_10.summary()


Wide Residual Network-28-10 created.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 16)           448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 16)           64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 32, 32, 16)          

In [ ]:
# from keras.optimizers import SGD
import keras
from tensorflow.keras.optimizers.legacy import SGD
from keras.datasets import mnist
import numpy as np

if __name__ == '__main__':

    num_classes = 10
    batch_size = 128

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    key_len = 20              ## desired WM key length
    embed_lr = 0.0008
    p_threshold = 0.0001
    embed_epoch = 2

    ## ---- Embed WM ------ ##
    model = create_model()
    model.load_weights('/content/drive/MyDrive/Colab Notebooks/markedWeights.h5')
    model.compile(loss='categorical_crossentropy',
                optimizer=SGD(learning_rate=embed_lr, momentum=0.9, decay=0.0, nesterov=True), metrics=['accuracy'])
    X_key, Y_key = key_generation(x_train, y_train, model, key_len, num_classes, embed_epoch)


    print("Detect WM")
    ## ----- Detect WM ------ ##
    #marked_model = create_model()
    #marked_model.load_weights('/content/drive/MyDrive/Colab Notebooks/markedWeights'+'.h5')
    #marked_model.compile(loss='categorical_crossentropy',
                #optimizer=SGD(learning_rate=embed_lr, momentum=0.9, decay=0.0, nesterov=True), metrics=['accuracy'])
    #preds_onehot = marked_model.predict(X_key, batch_size=batch_size)
    preds_onehot = model.predict(X_key, batch_size=batch_size)
    Y_preds = np.reshape(np.argmax(preds_onehot, axis=1), (key_len, 1))
    m = count_response_mismatch(Y_preds, Y_key)
    theta = compute_mismatch_threshold(C=num_classes, Kp=key_len, p=p_threshold) # pk = 1/C, |K|: # trials

    print('Probability threshold p is ', p_threshold)
    print('Mismatch threshold is : ', theta)
    print('Mismatch count of marked model on WM key set = ', m)
    print("If the marked model is correctly authenticated by the owner: ", m < theta)


11490434/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               401920    
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669706 (2.55 MB)
Trainable params: 669706 (2.55 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________